In [1]:
from pyzotero import zotero
from zotero2qmd.zotero2qmd import item2main, main2qmd, filter_pubs
from collections import Counter
import re
import yaml
from pathlib import Path

In [2]:
keypath = Path(".zotero")
with keypath.open() as keyfile:
    key_text = keyfile.readline().strip()

In [3]:
zot = zotero.Zotero(library_id='7642731', library_type='user', api_key=key_text)

In [4]:
pubs = zot.publications()

In [5]:
filtered_pubs = filter_pubs(pubs)

In [6]:
all_mains = [item2main(x) for x in filtered_pubs]

In [7]:
all_mains[0]

{'params': {'key': 'M3E7A7LS', 'notes': ''},
 'author': [{'name': {'given': 'Ingrid', 'family': 'Rosenfelder'}},
  {'name': {'given': 'Josef', 'family': 'Fruehwald'}},
  {'name': {'given': 'Keelan', 'family': 'Evanini'}},
  {'name': {'given': 'Jiahong', 'family': 'Yuan'}}],
 'title': 'FAVE (Forced Alignment and Vowel Extraction) Program Suite.',
 'date': '2011',
 'date-format': 'YYYY',
 'description': 'v0.0.0',
 'abstract': '',
 'citation': {'type': 'software',
  'issued': '2011',
  'url': 'http://fave.ling.upenn.edu/',
  'version': 'v0.0.0'}}

In [8]:
def make_file_names(all_mains):
    first_aut = [x["author"][0]["name"]["family"] for x in all_mains]
    years = [x["date"].split("-")[0] for x in all_mains]
    keys = [x["params"]["key"] for x in all_mains]

    all_stem = [f"{aut}_{year}_{key}" for aut, year, init in zip(first_aut, years, keys)]

    all_stem = [re.sub(r"_$", "", x) for x in all_stem]
    return all_stem

In [9]:
all_stems = make_file_names(all_mains)

In [11]:
all_stems

["Rosenfelder_2011_['M3E7A7LS', 'K57PJ2RE', 'TRXWWXD2', 'GY5I9QMZ', 'URYFVWHE', 'GH2LZM4Y', 'LIGPGR5R', 'U5C56KNR', 'QZP9CAP8', 'W28LR5KT', 'LPQN5R3M', 'H347UNLS', 'ETG527QQ', '7HNEQZ7H', 'IZLBZKZV', 'MG8DFJEK', '733Q4MMI', 'TCTQU8FS', 'FU9RXCUV', '8ERVHBSG', 'TJI9T9W6', '258NF56V', '34D4IRNY', 'H3BZPAT7']",
 "Rosenfelder_2015_['M3E7A7LS', 'K57PJ2RE', 'TRXWWXD2', 'GY5I9QMZ', 'URYFVWHE', 'GH2LZM4Y', 'LIGPGR5R', 'U5C56KNR', 'QZP9CAP8', 'W28LR5KT', 'LPQN5R3M', 'H347UNLS', 'ETG527QQ', '7HNEQZ7H', 'IZLBZKZV', 'MG8DFJEK', '733Q4MMI', 'TCTQU8FS', 'FU9RXCUV', '8ERVHBSG', 'TJI9T9W6', '258NF56V', '34D4IRNY', 'H3BZPAT7']",
 "Rosenfelder_2022_['M3E7A7LS', 'K57PJ2RE', 'TRXWWXD2', 'GY5I9QMZ', 'URYFVWHE', 'GH2LZM4Y', 'LIGPGR5R', 'U5C56KNR', 'QZP9CAP8', 'W28LR5KT', 'LPQN5R3M', 'H347UNLS', 'ETG527QQ', '7HNEQZ7H', 'IZLBZKZV', 'MG8DFJEK', '733Q4MMI', 'TCTQU8FS', 'FU9RXCUV', '8ERVHBSG', 'TJI9T9W6', '258NF56V', '34D4IRNY', 'H3BZPAT7']",
 "Fruehwald_2023_['M3E7A7LS', 'K57PJ2RE', 'TRXWWXD2', 'GY5I9QMZ', 'URY

In [10]:
out_path = Path("research","papers")

In [10]:
for stem, item in zip(all_stems, all_mains):
    out_file = out_path.joinpath(stem).with_suffix(".qmd")
    if not out_file.exists():
        with out_file.open(mode = "w"):
            qmd_string = "---\n"+yaml.dump(item)+"\n---"
            out_file.write_text(qmd_string)